In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib.dates as mdates
from pathlib import Path

# Directorio de trabajo con los archivos Excel
input_dir = Path(r'C:\Users\luisg\OneDrive\Documentos\Proyecto_Fiverr\src\data\4759_PF4_A203_M12')

# Crear la carpeta de salida si no existe
output_base_dir = Path(r'C:\Users\luisg\OneDrive\Documentos\Proyecto_Fiverr\output\4759_PF4_A203_M12')
output_base_dir.mkdir(parents=True, exist_ok=True)

# Procesar cada archivo Excel en la carpeta
for file_path in input_dir.glob('*.xlsx'):
    # Cargar la hoja del archivo xlsx en un DataFrame usando pandas y openpyxl, sin encabezado
    df = pd.read_excel(file_path, sheet_name='Sheet1', header=None)

    # Eliminar las columnas que solo contienen NaN
    df = df.dropna(axis=1, how='all')

    # Eliminar las filas que contienen solo NaN
    df = df.dropna(how='all')

    # Convertir todos los datos a numérico, excepto la primera y tercera columna
    for col in df.columns[[1, 3]]:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Eliminar las filas que contienen solo NaN después de la conversión
    df = df.dropna(how='all')

    # Asegurar que la primera columna sea de tipo fecha
    df[0] = pd.to_datetime(df[0], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

    # Eliminar filas con fechas inválidas
    df = df.dropna(subset=[0])

    # Verificar si todos los valores en la tercera columna son iguales
    if df[2].nunique() == 1:
        graph_title = str(df[2].iloc[0])
    else:
        graph_title = "Proporción de OK y NOT OK cada 30 minutos"

    # Contar OK y NOT OK por fecha
    df['OK'] = (df[1] == 0).astype(int)
    df['NOT_OK'] = (df[1] == 2).astype(int)

    # Agrupar por intervalos de 30 minutos
    df['interval'] = df[0].dt.floor('30min')
    df_counts = df.groupby('interval')[['OK', 'NOT_OK']].sum().reset_index()

    # Calcular el total y los porcentajes
    df_counts['Total'] = df_counts['OK'] + df_counts['NOT_OK']
    df_counts['OK_Percent'] = df_counts['OK'] / df_counts['Total'] * 100
    df_counts['NOT_OK_Percent'] = df_counts['NOT_OK'] / df_counts['Total'] * 100

    # Establecer la fecha como índice
    df_counts = df_counts.set_index('interval')

    # Crear la figura y los ejes
    fig, ax = plt.subplots(figsize=(20, 10))

    # Graficar barras apiladas con porcentajes
    bar_width = pd.Timedelta(minutes=25)  # Ancho de las barras reducido para crear separación
    ax.bar(df_counts.index, df_counts['OK_Percent'], label='OK', color='green', alpha=0.7, width=bar_width)
    ax.bar(df_counts.index, df_counts['NOT_OK_Percent'], bottom=df_counts['OK_Percent'], label='NOT OK', color='red', alpha=0.7, width=bar_width)

    ax.set_xlabel('Fecha y Hora')
    ax.set_ylabel('Porcentaje')
    ax.set_title(graph_title)
    ax.legend(loc='upper left')

    # Configurar el eje X
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    plt.xticks(rotation=45, ha='right')

    # Configurar el eje Y
    ax.set_ylim(0, 105)  # Ajustar el límite superior del eje Y
    ax.yaxis.set_ticks(range(0, 101, 10))  # Establecer las marcas del eje Y
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Ajustar el diseño y los márgenes
    plt.tight_layout()

    # Crear la ruta de salida específica para el archivo actual
    output_sub_dir = output_base_dir / file_path.stem
    output_sub_dir.mkdir(parents=True, exist_ok=True)

    # Guardar la figura
    output_file_path = output_sub_dir / 'Grafica_tiempo_OK_NOTOK.png'
    plt.savefig(output_file_path, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Gráfico generado y guardado como '{output_file_path}'.")


Gráfico generado y guardado como 'C:\Users\luisg\OneDrive\Documentos\Proyecto_Fiverr\output\4759_PF4_A203_M12\188_a\Grafica_tiempo_OK_NOTOK.png'.
Gráfico generado y guardado como 'C:\Users\luisg\OneDrive\Documentos\Proyecto_Fiverr\output\4759_PF4_A203_M12\188_b\Grafica_tiempo_OK_NOTOK.png'.
Gráfico generado y guardado como 'C:\Users\luisg\OneDrive\Documentos\Proyecto_Fiverr\output\4759_PF4_A203_M12\188_c\Grafica_tiempo_OK_NOTOK.png'.


In [ ]:
"""
Creación de Gráficos de Análisis

Este script genera gráficos de dispersión y histogramas a partir de archivos de datos.

Imports:
    - pandas: Librería para manipulación de datos.
    - matplotlib.pyplot: Librería para creación de gráficos.
    - os: Librería para interactuar con el sistema operativo.
    - gc: Librería para la recolección de basura.
    - random: Librería para generar números aleatorios.

Funciones:
    - graficos_analisis: Genera gráficos de dispersión e histogramas.
"""

import pandas as pd
import matplotlib.pyplot as plt
import os
import gc
import random

def graficos_analisis(df, tipo, nombre_archivo, output_dir, max_graficos=50):
    """
    Crea gráficos de dispersión y histogramas.

    Args:
        df (DataFrame): DataFrame con los datos.
        tipo (str): Tipo de datos (OK o NOT OK).
        nombre_archivo (str): Nombre del archivo de datos.
        output_dir (str): Directorio de salida para los gráficos.
        max_graficos (int): Número máximo de gráficos a crear.
    """
    num_columns = df.shape[1]
    
    angulos = []
    pares = []
    
    # Seleccionar aleatoriamente hasta 50 pares de columnas
    column_pairs = list(range(0, num_columns, 2))
    random.shuffle(column_pairs)
    selected_pairs = column_pairs[:max_graficos]
    
    for i in selected_pairs:
        # Seleccionar columnas de par y ángulo (cambiando el orden)
        col_par = df.iloc[1:, i].dropna()
        col_angulo = df.iloc[1:, i+1].dropna() if i + 1 < num_columns else pd.Series(dtype='float64')
        
        # Asegurar que ambas columnas tengan el mismo tamaño
        if len(col_par) != len(col_angulo):
            min_length = min(len(col_par), len(col_angulo))
            col_par = col_par.iloc[:min_length]
            col_angulo = col_angulo.iloc[:min_length]
        
        # Almacenar valores de par y ángulo
        pares.extend(col_par)
        angulos.extend(col_angulo)
        
        # Gráfico de dispersión
        plt.figure(figsize=(10, 6))
        plt.scatter(col_par, col_angulo, alpha=0.5)
        plt.xlabel('Ángulo')
        plt.ylabel('Par')
        plt.title(f'Gráfico de dispersión de Par vs Ángulo ({tipo})')
        plt.grid(True)
        plt.savefig(f'{output_dir}/dispersión_{tipo}_{nombre_archivo}_columna_{i//2}.png')
        plt.close()
        
        # Liberar memoria
        del col_par, col_angulo
        gc.collect()
    
    # Histogramas
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    plt.hist(pares, bins=20, alpha=0.7)
    plt.xlabel('Ángulo')
    plt.ylabel('Frecuencia')
    plt.title(f'Histograma de Par ({tipo})')
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.hist(angulos, bins=20, alpha=0.7)
    plt.xlabel('Par')
    plt.ylabel('Frecuencia')
    plt.title(f'Histograma de Ángulo ({tipo})')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/histogramas_{tipo}_{nombre_archivo}.png')
    plt.close()
    
    # Liberar memoria
    del angulos, pares
    gc.collect()

# Crear carpeta para guardar las imágenes
output_base_dir = 'output'
os.makedirs(output_base_dir, exist_ok=True)

# Procesar todos los archivos .xlsx en la carpeta 'src/data' y sus subcarpetas
data_base_dir = 'src/data'

for root, dirs, files in os.walk(data_base_dir):
    for file_name in files:
        if file_name.endswith('.xlsx'):
            file_path = os.path.join(root, file_name)
            relative_folder = os.path.relpath(root, data_base_dir)
            output_dir = os.path.join(output_base_dir, f"{relative_folder}_angulovspar")
            
            os.makedirs(output_dir, exist_ok=True)
            
            # Leer el archivo Excel
            df = pd.read_excel(file_path, header=None)
            df = df.drop(df.columns[[0, 2]], axis=1)
            # Transponer el DataFrame
            df_transposed = df.transpose()
            
            # Filtrar columnas OK (primer valor es 0)
            columnas_ok = df_transposed.columns[df_transposed.iloc[0] == 0]
            
            # Filtrar columnas NOT OK (primer valor es 2)
            columnas_not_ok = df_transposed.columns[df_transposed.iloc[0] == 2]
            
            # Crear DataFrames filtrados
            df_ok = df_transposed[columnas_ok]
            df_not_ok = df_transposed[columnas_not_ok]
            
            # Obtener el nombre del archivo sin la extensión
            nombre_archivo = os.path.splitext(file_name)[0]
            
            # Crear gráficos para atornillados OK (máximo 50)
            graficos_analisis(df_ok, 'OK', nombre_archivo, output_dir, max_graficos=50)
            
            # Crear gráficos para atornillados NOT OK (máximo 50)
            graficos_analisis(df_not_ok, 'NOT OK', nombre_archivo, output_dir, max_graficos=50)
            
            # Liberar memoria
            del df, df_transposed, df_ok, df_not_ok
            gc.collect()

print("Procesamiento completado. Los gráficos se han guardado en la carpeta 'output'.")
